<a href="https://colab.research.google.com/github/nedazar/NLP-Assignment-2---Group-15/blob/main/Q6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install checklist
#jupyter nbextension install --py --sys-prefix checklist.viewer
#jupyter nbextension enable --py --sys-prefix checklist.viewer

In [4]:
!pip install simpletransformers


     |████████████████████████████████| 249 kB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 47.6 MB/s 
     |████████████████████████████████| 1.8 MB 40.0 MB/s 
     |████████████████████████████████| 10.1 MB 20.4 MB/s 
     |████████████████████████████████| 342 kB 40.4 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 145 kB 35.1 MB/s 
     |████████████████████████████████| 181 kB 49.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.3 MB/s 
     |████████████████████████████████| 136 kB 43.4 MB/s 
     |████████████████████████████████| 212 kB 46.0 MB/s 
     |████████████████████████████████| 127 kB 46.9 MB/s 
     |████████████████████████████████| 271 kB 48.5 MB/s 
     |████████████████████████████████| 144 kB 50.0 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 111 kB 51.2 MB/s 
     |██████████████

In [1]:
from checklist.editor import Editor
from checklist.perturb import Perturb
import spacy
from google.colab import drive, files
import pandas as pd
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs


In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
# mount file path to google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# LOAD SUBSET DIAGNOSTIC TEST
olid_diag_test = pd.read_csv('/content/drive/MyDrive/VU/NLPT/Assignment 2/olid-subset-diagnostic-tests.csv')
olid_diag_test = olid_diag_test.drop(columns = ['id'])
olid_diag_test.head(100)

,text,labels
0,@USER @USER Who the hell does he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 lost my acca on the first fucking fight cba,1
...,...,...
95,@USER He is obviously getting suspended. He is...,0
96,#Canada - EXCLUSIVE: #Trudeau #Liberals leave ...,0
97,@USER @USER ...than why did you show us how ho...,0
98,@USER @USER @USER You have yet to answer what ...,0


In [6]:
pdata = list(nlp.pipe(olid_diag_test['text']))


In [7]:
ret = Perturb.perturb(pdata, Perturb.add_negation)


In [8]:
ret.data

[['@USER @USER Who the hell does he think he is?',
  "@USER @USER Who the hell doesn't he think he is?"],
 ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
  "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL"],
 ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
  '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'],
 ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes.

In [9]:
# Load saved TRAINED model
model = torch.load('/content/drive/MyDrive/VU/NLPT/Assignment 2/saved_model')

In [ ]:
# Create prediction for every sentence negated sentence
test_set_predictions = []

for text in ret.data:
  test_set_predictions.append(model.predict(text[1])[0][0])
  


In [ ]:
test_set_predictions

In [ ]:
# Create prediction for every sentence negated sentence
prediction_after_negation = []

for negated_text in ret.data:
  prediction_after_negation.append((negated_text,model.predict(negated_text[1])[0][0]))
  
  

In [24]:
for prediction in prediction_after_negation:
  print("Original Sentence: ",prediction[0][0])
  print("Negated Sentence : ",prediction[0][1])
  print("New prediction   : ",prediction[1])
  print()

Original Sentence:  @USER @USER Who the hell does he think he is?
Negated Sentence :  @USER @USER Who the hell doesn't he think he is?
New prediction   :  1

Original Sentence:  #BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL
Negated Sentence :  #BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL
New prediction   :  0

Original Sentence:  #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!
Negated Sentence :  #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good ridda